## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization, Activation
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\Jason\Anaconda3\envs\tensorFlowGpu\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:
input_shape = (32, 32, 3)
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(64, (3, 3), padding='same', input_shape=input_shape))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(Convolution2D(64, (3, 3), padding='same'))
classifier.add(Activation('relu'))
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64, (3, 3), padding='same'))
classifier.add(Convolution2D(64, (3, 3), padding='same'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(96, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=64,epochs=10)

C:\Users\Jason\Anaconda3\envs\tensorFlowGpu\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/10
50000/50000 [==============================] - 24s 474us/step - loss: 1.4416 - acc: 0.5039
Epoch 2/10
50000/50000 [==============================] - 22s 431us/step - loss: 1.0089 - acc: 0.6456
Epoch 3/10
50000/50000 [==============================] - 22s 431us/step - loss: 0.8927 - acc: 0.6871
Epoch 4/10
50000/50000 [==============================] - 23s 456us/step - loss: 0.8262 - acc: 0.7104
Epoch 5/10
50000/50000 [==============================] - 22s 446us/step - loss: 0.7719 - acc: 0.7292
Epoch 6/10
50000/50000 [==============================] - 23s 460us/step - loss: 0.7149 - acc: 0.7516
Epoch 7/10
50000/50000 [==============================] - 23s 450us/step - loss: 0.6591 - acc: 0.7688
Epoch 8/10
50000/50000 [==============================] - 22s 437us/step - loss: 0.6016 - acc: 0.7902
Epoch 9/10
50000/50000 [==============================] - 22s 439us/step - loss: 0.5517 - acc: 0.8084
Epoch 10/10
50000/50000 [==============================] - 22s 432us/step - loss: 

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[0.068276  , 0.00227719, 0.3603254 , 0.25846002, 0.09127458,
        0.14020912, 0.02366552, 0.04623214, 0.00330754, 0.00597249]],
      dtype=float32)